In [4]:
"""
    generate_log_spaced_times(L, z, num_points, regions; ...)

Generates logarithmically spaced measurement times, snapping them to interval-based
grids defined by different regions.

# Arguments
- `regions`: A vector of tuples `(t_threshold, interval)`. Each tuple defines that
  for times `t < t_threshold`, the measurement times will be multiples of `interval`.
  The regions should be sorted by `t_threshold`. The last threshold should be `Inf`.

# Example
`regions = [(100, 1), (1000, 10), (Inf, 100)]` means:
- `t < 100`: interval is 1 (measure every step)
- `100 <= t < 1000`: interval is 10
- `t >= 1000`: interval is 100
"""
function generate_log_spaced_times(
    L::Int, 
    z::Float64, 
    num_points::Int,
    regions::Vector; 
    max_scaled_time::Float64=0.5, 
    t_min::Int=1
)
    t_max = max_scaled_time * (L^z)
    if t_max < t_min
        return [t_min]
    end

    # 1. Generate the "ideal" logarithmically spaced points
    log_steps = range(log10(t_min), stop=log10(t_max), length=num_points)
    tmea_float = 10 .^ log_steps

    snapped_times = Int[]
    for t in tmea_float
        # 2. For each ideal point, find which region it falls into
        interval = 1
        for (threshold, intrvl) in regions
            if t < threshold
                interval = intrvl
                break
            end
        end
        
        # 3. Snap the ideal point to the nearest multiple of the determined interval
        snapped_t = round(Int, t / interval) * interval
        
        # Ensure the snapped time is at least t_min
        push!(snapped_times, max(t_min, snapped_t))
    end

    # 4. Return the unique, sorted list of valid measurement times
    return sort(unique(snapped_times))
end

z = 2.7
L = 200
total_points = 200
regions = [
        (100, 1),
        (1000, 10),
        (Inf, 100)
    ]
tmea = generate_log_spaced_times(L, z, total_points,regions; max_scaled_time = 0.5)

generate_log_spaced_times

In [ ]:
z = 2.7
L = 200
total_points = 200
regions = [
        (100, 1),
        (1000, 10),
        (Inf, 100)
    ]
tmea = generate_log_spaced_times(L, z, total_points,regions; max_scaled_time = 0.5)

173-element Vector{Int64}:
      1
      2
      3
      4
      5
      6
      7
      8
      9
     10
      ⋮
 472200
 505600
 541400
 579700
 620800
 664700
 711800
 762200
 816100

In [1]:
"""
    generate_log_spaced_times(...)

Generates logarithmically spaced measurement times, with options to select a specific
sub-region for execution.

# Arguments
- `regions`: A vector of tuples `(t_threshold, interval)`.
- `region_index` (optional): An integer `n` specifying which region to generate times for.
  If `nothing` (the default), times are generated for all regions.
"""
function generate_log_spaced_times(
    L::Int, 
    z::Float64, 
    num_points::Int,
    regions::Vector; 
    max_scaled_time::Float64=0.5, 
    t_min::Int=1,
    region_index::Union{Int, Nothing}=nothing
)
    # Determine the absolute maximum time for the whole simulation
    t_max_global = max_scaled_time * (L^z)
    if t_max_global < t_min
        return t_min > 0 ? [t_min] : Int[]
    end

    # 1. Determine the time boundaries [t_start, t_end) for the selected region
    local_t_start = t_min
    local_t_end = t_max_global

    if region_index !== nothing
        if !(1 <= region_index <= length(regions))
            error("region_index must be between 1 and $(length(regions)), or be `nothing`.")
        end
        
        # The end of the current region is its threshold
        local_t_end = regions[region_index][1]

        # The start of the current region is the threshold of the previous one
        if region_index > 1
            local_t_start = regions[region_index - 1][1]
        end
    end

    # 2. Generate the "ideal" log-spaced points across the ENTIRE global range
    #    This is crucial to maintain the correct logarithmic density of points.
    log_steps = range(log10(t_min), stop=log10(t_max_global), length=num_points)
    tmea_float = 10 .^ log_steps

    snapped_times = Int[]
    for t in tmea_float
        # 3. Filter out points that are not in our selected region
        if t < local_t_start || t >= local_t_end
            continue
        end

        # 4. Find the correct interval for this point's region
        current_interval = 1
        for (threshold, intrvl) in regions
            if t < threshold
                current_interval = intrvl
                break
            end
        end
        
        # 5. Snap the ideal point to the nearest multiple of the interval
        snapped_t = round(Int, t / current_interval) * current_interval
        
        # Ensure the snapped time is valid and within the region's bounds
        if snapped_t >= local_t_start
             push!(snapped_times, max(t_min, snapped_t))
        end
    end

    # 6. Return the unique, sorted list of valid measurement times
    return sort(unique(snapped_times))
end


# In your main() function in main.jl

# --- Configuration ---
z = 2.7
total_points = 500 # You might want more total points for better resolution
regions = [
    (100, 1),
    (1000, 10),
    (Inf, 100)
]

# This would typically come from a command-line argument, e.g., ARGS[7]
# For this example, let's hardcode it to run the 2nd region.
target_region = 2 
interval = regions[target_region][2]
@show   interval
# --- Generate tmea for the specified region ---
tmea = Int[]
needs_time_obs = true # Your logic here

if needs_time_obs
    println("Generating measurement times for region #$(target_region)...")
    tmea = generate_log_spaced_times(L, z, total_points, regions; 
                                    max_scaled_time=0.5,
                                    region_index=target_region)
end


interval = 10
Generating measurement times for region #2...


UndefVarError: UndefVarError: `L` not defined in `Main`
Suggestion: check for spelling errors or missing imports.